In [10]:
from dataloader import get_whole_data , get_whole_data_avg
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler , PolynomialFeatures, OrdinalEncoder
from sklearn.model_selection import train_test_split
from model import *
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import r2_score, mean_absolute_error , mean_squared_error


In [11]:
def prepare_data(col_to_drop=None):
  data = get_whole_data()
  drop_col = ['Installation_Year','Replicates']
  data.columns = [c.replace(' ', '_') for c in data.columns]
  if col_to_drop is not None and col_to_drop != 'Sheeting_Type':
    drop_col.append(col_to_drop)
  data = data.drop(drop_col,axis=1)
  threeM = data.query('Brand == "3M"')
  threeM_dia = threeM.query('Sheeting_Type == "Diamond"')
  threeM_hip = threeM.query('Sheeting_Type == "HIP"')
  threeM_eng = threeM.query('Sheeting_Type == "Engineering Grade"')
  aviery = data.query('Brand == "Avery Dennison"')
  aviery_dia = aviery.query('Sheeting_Type == "Diamond"')
  aviery_hip = aviery.query('Sheeting_Type == "HIP"')
  aviery_eng = aviery.query('Sheeting_Type == "Engineering Grade"')
  tong = data.query('Brand == "TongMing"')
  tong_dia = tong.query('Sheeting_Type == "Diamond"')
  tong_hip = tong.query('Sheeting_Type == "HIP"')
  tong_eng = tong.query('Sheeting_Type == "Engineering Grade"')
  train = pd.concat([threeM_dia,threeM_hip,aviery_eng,tong_hip,tong_eng])
  train = pd.concat([threeM_dia,threeM_hip,aviery_eng,tong_hip,tong_eng])

  validation =  pd.concat([threeM_eng.iloc[960//2:],aviery_hip,tong_dia.iloc[960//2:]])
  test =pd.concat([threeM_eng.iloc[:960//2],aviery_dia,tong_dia.iloc[:960//2]])
  if col_to_drop == 'Sheeting_Type':
    train = train.drop(['Brand','Sheeting_Type'],axis=1)
    validation = validation.drop(['Brand','Sheeting_Type'],axis=1)
    test = test.drop(['Brand','Sheeting_Type'],axis=1)
  else:
    train = train.drop(['Brand',],axis=1)
    validation = validation.drop(['Brand'],axis=1)
    test = test.drop(['Brand'],axis=1)
  return train, validation, test

def exp_ohe(col_to_drop=None):
  train , validation , test = prepare_data(col_to_drop)
  if col_to_drop == None:
    t_x = train.iloc[:,0:5]  #independent columns
    t_y = train.iloc[:,-1]    #target column 
    v_x = validation.iloc[:,0:5]  #independent columns
    v_y = validation.iloc[:,-1]    #target column 
    te_x = test.iloc[:,0:5]  #independent columns
    te_y = test.iloc[:,-1]    #target column 
  else:
    t_x = train.iloc[:,0:4]  #independent columns
    t_y = train.iloc[:,-1]    #target column 
    v_x = validation.iloc[:,0:4]  #independent columns
    v_y = validation.iloc[:,-1]    #target column 
    te_x = test.iloc[:,0:4]  #independent columns
    te_y = test.iloc[:,-1]    #target column 
  oe = OneHotEncoder()
  oe.fit(t_x.to_numpy()[:,:-1])
  scaler = StandardScaler()

  x_enc = oe.transform(t_x.to_numpy()[:,:-1]).toarray()
  x_enc = np.append(x_enc, t_x.to_numpy()[:,-1].reshape(len(t_x),1) / 10, axis=1).astype(np.float32)

  y_scaled = scaler.fit_transform(t_y.to_numpy().reshape(-1,1)).astype(np.float32)
  #VALID
  vx_enc = oe.transform(v_x.to_numpy()[:,:-1]).toarray()
  vx_enc = np.append(vx_enc, v_x.to_numpy()[:,-1].reshape(len(v_x),1) / 10, axis=1).astype(np.float32)
  vy_scaled = scaler.fit_transform(v_y.to_numpy().reshape(-1,1)).astype(np.float32)
  #TEST
  tex_enc = oe.transform(te_x.to_numpy()[:,:-1]).toarray()
  tex_enc = np.append(tex_enc, te_x.to_numpy()[:,-1].reshape(len(te_x),1) / 10, axis=1).astype(np.float32)
  tey_scaled = scaler.fit_transform(te_y.to_numpy().reshape(-1,1)).astype(np.float32)
  model = create_onehotencode_model2(input=x_enc.shape[-1])
  es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=2)
  rlr = ReduceLROnPlateau(patience=1,min_lr=0.00001)
  history = model.fit(x_enc,y_scaled, validation_data=(vx_enc, vy_scaled) , batch_size=100,epochs=500,callbacks=[rlr,es],verbose=0)
  #TEST RESULTS
  prediction = model.predict(tex_enc)
  prediction_after_scale = scaler.inverse_transform(prediction)
  true_values = scaler.inverse_transform(tey_scaled)
  test_r2 = r2_score(true_values,prediction_after_scale)
  test_mae = mean_absolute_error(true_values,prediction_after_scale)
  #TRAIN RESULTS
  prediction = model.predict(x_enc)
  prediction_after_scale = scaler.inverse_transform(prediction)
  true_values = scaler.inverse_transform(y_scaled)
  train_r2 = r2_score(true_values,prediction_after_scale)
  train_mae = mean_absolute_error(true_values,prediction_after_scale)

  #VALIDATION RESULTS
  prediction = model.predict(vx_enc)
  prediction_after_scale = scaler.inverse_transform(prediction)
  true_values = scaler.inverse_transform(vy_scaled)
  valid_r2 = r2_score(true_values,prediction_after_scale)
  valid_mae = mean_absolute_error(true_values,prediction_after_scale)
  return (train_r2,valid_r2,test_r2) , (train_mae,valid_mae,test_mae)

def exp_linear(col_to_drop=None):
  train , validation , test = prepare_data(col_to_drop)
  if col_to_drop == None:
    t_x = train.iloc[:,0:5]  #independent columns
    t_y = train.iloc[:,-1]    #target column 
    v_x = validation.iloc[:,0:5]  #independent columns
    v_y = validation.iloc[:,-1]    #target column 
    te_x = test.iloc[:,0:5]  #independent columns
    te_y = test.iloc[:,-1]    #target column 
  else:
    t_x = train.iloc[:,0:4]  #independent columns
    t_y = train.iloc[:,-1]    #target column 
    v_x = validation.iloc[:,0:4]  #independent columns
    v_y = validation.iloc[:,-1]    #target column 
    te_x = test.iloc[:,0:4]  #independent columns
    te_y = test.iloc[:,-1]    #target column 
  oe = OneHotEncoder()
  oe.fit(t_x.to_numpy()[:,:-1])
  scaler = StandardScaler()

  x_enc = oe.transform(t_x.to_numpy()[:,:-1]).toarray()
  x_enc = np.append(x_enc, t_x.to_numpy()[:,-1].reshape(len(t_x),1) / 10, axis=1).astype(np.float32)

  y_scaled = scaler.fit_transform(t_y.to_numpy().reshape(-1,1)).astype(np.float32)
  #VALID
  vx_enc = oe.transform(v_x.to_numpy()[:,:-1]).toarray()
  vx_enc = np.append(vx_enc, v_x.to_numpy()[:,-1].reshape(len(v_x),1) / 10, axis=1).astype(np.float32)
  vy_scaled = scaler.fit_transform(v_y.to_numpy().reshape(-1,1)).astype(np.float32)
  #TEST
  tex_enc = oe.transform(te_x.to_numpy()[:,:-1]).toarray()
  tex_enc = np.append(tex_enc, te_x.to_numpy()[:,-1].reshape(len(te_x),1) / 10, axis=1).astype(np.float32)
  tey_scaled = scaler.fit_transform(te_y.to_numpy().reshape(-1,1)).astype(np.float32)
  
  model = create_linear_model(x_enc.shape[-1])
  es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=2)
  rlr = ReduceLROnPlateau(patience=1,min_lr=0.00001)
  history = model.fit(x_enc,y_scaled, validation_data=(vx_enc, vy_scaled) , batch_size=100,epochs=500,callbacks=[rlr,es],verbose=0)

  #TEST RESULTS
  prediction = model.predict(tex_enc)
  prediction_after_scale = scaler.inverse_transform(prediction)
  true_values = scaler.inverse_transform(tey_scaled)
  test_r2 = r2_score(true_values,prediction_after_scale)
  test_mae = mean_absolute_error(true_values,prediction_after_scale)

  #TRAIN RESULTS
  prediction = model.predict(x_enc)
  prediction_after_scale = scaler.inverse_transform(prediction)
  true_values = scaler.inverse_transform(y_scaled)
  train_r2 = r2_score(true_values,prediction_after_scale)
  train_mae = mean_absolute_error(true_values,prediction_after_scale)

  #VALIDATION RESULTS
  prediction = model.predict(vx_enc)
  prediction_after_scale = scaler.inverse_transform(prediction)
  true_values = scaler.inverse_transform(vy_scaled)
  valid_r2 = r2_score(true_values,prediction_after_scale)
  valid_mae = mean_absolute_error(true_values,prediction_after_scale)
  return (train_r2,valid_r2,test_r2) , (train_mae,valid_mae,test_mae)

def exp_poly(col_to_drop=None):
  train , validation , test = prepare_data(col_to_drop)
  if col_to_drop == None:
    t_x = train.iloc[:,0:5]  #independent columns
    t_y = train.iloc[:,-1]    #target column 
    v_x = validation.iloc[:,0:5]  #independent columns
    v_y = validation.iloc[:,-1]    #target column 
    te_x = test.iloc[:,0:5]  #independent columns
    te_y = test.iloc[:,-1]    #target column 
  else:
    t_x = train.iloc[:,0:4]  #independent columns
    t_y = train.iloc[:,-1]    #target column 
    v_x = validation.iloc[:,0:4]  #independent columns
    v_y = validation.iloc[:,-1]    #target column 
    te_x = test.iloc[:,0:4]  #independent columns
    te_y = test.iloc[:,-1]    #target column 
  oe = OneHotEncoder()
  poly = PolynomialFeatures(2)
  oe.fit(t_x.to_numpy()[:,:-1])
  scaler = StandardScaler()

  x_enc = oe.transform(t_x.to_numpy()[:,:-1]).toarray()
  x_enc = np.append(x_enc, t_x.to_numpy()[:,-1].reshape(len(t_x),1) / 10, axis=1)
  x_enc = poly.fit_transform(x_enc).astype(np.float32)
  y_scaled = scaler.fit_transform(t_y.to_numpy().reshape(-1,1)).astype(np.float32)
  #VALID
  vx_enc = oe.transform(v_x.to_numpy()[:,:-1]).toarray()
  vx_enc = np.append(vx_enc, v_x.to_numpy()[:,-1].reshape(len(v_x),1) / 10, axis=1)
  vx_enc = poly.fit_transform(vx_enc).astype(np.float32)
  vy_scaled = scaler.fit_transform(v_y.to_numpy().reshape(-1,1)).astype(np.float32)
  #TEST
  tex_enc = oe.transform(te_x.to_numpy()[:,:-1]).toarray()
  tex_enc = np.append(tex_enc, te_x.to_numpy()[:,-1].reshape(len(te_x),1) / 10, axis=1)
  tex_enc = poly.fit_transform(tex_enc).astype(np.float32)
  tey_scaled = scaler.fit_transform(te_y.to_numpy().reshape(-1,1)).astype(np.float32)
  poly_input_size = vx_enc[0].shape[0]
  model = create_linear_model(poly_input_size)
  es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=2)
  rlr = ReduceLROnPlateau(patience=1,min_lr=0.00001)
  history = model.fit(x_enc,y_scaled, validation_data=(vx_enc, vy_scaled) , batch_size=100,epochs=500,callbacks=[rlr,es],verbose=0)

  #TEST RESULTS
  prediction = model.predict(tex_enc)
  prediction_after_scale = scaler.inverse_transform(prediction)
  true_values = scaler.inverse_transform(tey_scaled)
  test_r2 = r2_score(true_values,prediction_after_scale)
  test_mae = mean_absolute_error(true_values,prediction_after_scale)

  #TRAIN RESULTS
  prediction = model.predict(x_enc)
  prediction_after_scale = scaler.inverse_transform(prediction)
  true_values = scaler.inverse_transform(y_scaled)
  train_r2 = r2_score(true_values,prediction_after_scale)
  train_mae = mean_absolute_error(true_values,prediction_after_scale)

  #VALIDATION RESULTS
  prediction = model.predict(vx_enc)
  prediction_after_scale = scaler.inverse_transform(prediction)
  true_values = scaler.inverse_transform(vy_scaled)
  valid_r2 = r2_score(true_values,prediction_after_scale)
  valid_mae = mean_absolute_error(true_values,prediction_after_scale)
  return (train_r2,valid_r2,test_r2) , (train_mae,valid_mae,test_mae)

def exp_sepEmbed(col_to_drop=None):
  train , validation , test = prepare_data(col_to_drop)
  if col_to_drop == None:
    t_x = train.iloc[:,0:5]  #independent columns
    t_y = train.iloc[:,-1]    #target column 
    v_x = validation.iloc[:,0:5]  #independent columns
    v_y = validation.iloc[:,-1]    #target column 
    te_x = test.iloc[:,0:5]  #independent columns
    te_y = test.iloc[:,-1]    #target column 
  else:
    t_x = train.iloc[:,0:4]  #independent columns
    t_y = train.iloc[:,-1]    #target column 
    v_x = validation.iloc[:,0:4]  #independent columns
    v_y = validation.iloc[:,-1]    #target column 
    te_x = test.iloc[:,0:4]  #independent columns
    te_y = test.iloc[:,-1]    #target column 
  oe = OrdinalEncoder()
  oe.fit(t_x.to_numpy()[:,:-1])
  scaler = StandardScaler()

  x_enc = oe.transform(t_x.to_numpy()[:,:-1])#.toarray()
  x_enc = np.append(x_enc, t_x.to_numpy()[:,-1].reshape(len(t_x),1) / 10, axis=1).astype(np.float32)

  y_scaled = scaler.fit_transform(t_y.to_numpy().reshape(-1,1)).astype(np.float32)
  #VALID
  vx_enc = oe.transform(v_x.to_numpy()[:,:-1])
  vx_enc = np.append(vx_enc, v_x.to_numpy()[:,-1].reshape(len(v_x),1) / 10, axis=1).astype(np.float32)
  vy_scaled = scaler.fit_transform(v_y.to_numpy().reshape(-1,1)).astype(np.float32)
  #TEST
  tex_enc = oe.transform(te_x.to_numpy()[:,:-1])
  tex_enc = np.append(tex_enc, te_x.to_numpy()[:,-1].reshape(len(te_x),1) / 10, axis=1).astype(np.float32)
  tey_scaled = scaler.fit_transform(te_y.to_numpy().reshape(-1,1)).astype(np.float32)
  es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=2)
  rlr = ReduceLROnPlateau(patience=1,min_lr=0.00001)
  if col_to_drop == None:
    model = create_sep_embed_model()
    history = model.fit(x=[x_enc[:,0],x_enc[:,1],x_enc[:,2],x_enc[:,3],x_enc[:,4]],y=y_scaled,
 validation_data=([vx_enc[:,0],vx_enc[:,1],vx_enc[:,2],vx_enc[:,3],vx_enc[:,4]], vy_scaled) ,
  batch_size=100,epochs=500,callbacks=[rlr,es],verbose=0)
    #TEST
    test_prediction = model.predict(x=[tex_enc[:,0],tex_enc[:,1],tex_enc[:,2],tex_enc[:,3],tex_enc[:,4]])
    train_prediction = model.predict(x=[x_enc[:,0],x_enc[:,1],x_enc[:,2],x_enc[:,3],x_enc[:,4]])
    valid_prediction = model.predict(x=[vx_enc[:,0],vx_enc[:,1],vx_enc[:,2],vx_enc[:,3],vx_enc[:,4]])
  else:
    model = create_sep_embed_model_leave_one()
    history = model.fit(x=[x_enc[:,0],x_enc[:,1],x_enc[:,2],x_enc[:,3]],y=y_scaled,
      validation_data=([vx_enc[:,0],vx_enc[:,1],vx_enc[:,2],vx_enc[:,3]],vy_scaled) ,
       batch_size=100,epochs=500,callbacks=[rlr,es],verbose=0)
    #Test
    test_prediction = model.predict(x=[tex_enc[:,0],tex_enc[:,1],tex_enc[:,2],tex_enc[:,3]])
    train_prediction = model.predict(x=[x_enc[:,0],x_enc[:,1],x_enc[:,2],x_enc[:,3]])
    valid_prediction = model.predict(x=[vx_enc[:,0],vx_enc[:,1],vx_enc[:,2],vx_enc[:,3]])

  #TEST RESULTS
  prediction_after_scale = scaler.inverse_transform(test_prediction)
  true_values = scaler.inverse_transform(tey_scaled)
  test_r2 = r2_score(true_values,prediction_after_scale)
  test_mae = mean_absolute_error(true_values,prediction_after_scale)

  #TRAIN RESULTS
  prediction_after_scale = scaler.inverse_transform(train_prediction)
  true_values = scaler.inverse_transform(y_scaled)
  train_r2 = r2_score(true_values,prediction_after_scale)
  train_mae = mean_absolute_error(true_values,prediction_after_scale)

  #VALIDATION RESULTS
  prediction_after_scale = scaler.inverse_transform(valid_prediction)
  true_values = scaler.inverse_transform(vy_scaled)
  valid_r2 = r2_score(true_values,prediction_after_scale)
  valid_mae = mean_absolute_error(true_values,prediction_after_scale)
  return (train_r2,valid_r2,test_r2) , (train_mae,valid_mae,test_mae)

def exp_sharedEmbed(col_to_drop=None):
  train , validation , test = prepare_data(col_to_drop)
  if col_to_drop == None:
    t_x = train.iloc[:,0:5]  #independent columns
    t_y = train.iloc[:,-1]    #target column 
    v_x = validation.iloc[:,0:5]  #independent columns
    v_y = validation.iloc[:,-1]    #target column 
    te_x = test.iloc[:,0:5]  #independent columns
    te_y = test.iloc[:,-1]    #target column 
  else:
    t_x = train.iloc[:,0:4]  #independent columns
    t_y = train.iloc[:,-1]    #target column 
    v_x = validation.iloc[:,0:4]  #independent columns
    v_y = validation.iloc[:,-1]    #target column 
    te_x = test.iloc[:,0:4]  #independent columns
    te_y = test.iloc[:,-1]    #target column 
  x_before = t_x.astype('str').to_numpy()[:,:-1]
  unique_values = pd.unique(t_x.astype('str').values.ravel()) 
  if col_to_drop is None:
    oe = OrdinalEncoder(categories=[unique_values]*4)
  else:
    oe = OrdinalEncoder(categories=[unique_values]*3)
  
  x_enc = oe.fit_transform(x_before)
  scaler = StandardScaler()

  x_enc = np.append(x_enc, t_x.to_numpy()[:,-1].reshape(len(t_x),1) / 10, axis=1).astype(np.float32)
  y_scaled = scaler.fit_transform(t_y.to_numpy().reshape(-1,1)).astype(np.float32)

  #VALID
  vx_before = v_x.astype('str').to_numpy()[:,:-1]
  vx_enc = oe.transform(vx_before)
  vx_enc = np.append(vx_enc, v_x.to_numpy()[:,-1].reshape(len(v_x),1) / 10, axis=1).astype(np.float32)
  vy_scaled = scaler.fit_transform(v_y.to_numpy().reshape(-1,1)).astype(np.float32)
  #TEST
  tex_before = te_x.astype('str').to_numpy()[:,:-1]
  tex_enc = oe.transform(tex_before)
  tex_enc = np.append(tex_enc, te_x.to_numpy()[:,-1].reshape(len(te_x),1) / 10, axis=1).astype(np.float32)
  tey_scaled = scaler.fit_transform(te_y.to_numpy().reshape(-1,1)).astype(np.float32)

  es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=2)
  rlr = ReduceLROnPlateau(patience=1,min_lr=0.00001)
  if col_to_drop == None:
    model = create_embed_model(input_dim=4)
    history = model.fit(x=[x_enc[:,:-1],x_enc[:,-1]],y=y_scaled,
 validation_data=([vx_enc[:,:-1],vx_enc[:,-1]], vy_scaled) , batch_size=100,epochs=500,callbacks=[rlr,es],verbose=0)
    #TEST
  else:
    model = create_embed_model(input_dim=3)
    history = model.fit(x=[x_enc[:,:-1],x_enc[:,-1]],y=y_scaled,
 validation_data=([vx_enc[:,:-1],vx_enc[:,-1]], vy_scaled) , batch_size=100,epochs=500,callbacks=[rlr,es],verbose=0)
  test_prediction = model.predict(x=[tex_enc[:,:-1],tex_enc[:,-1]])
  train_prediction = model.predict(x=[x_enc[:,:-1],x_enc[:,-1]])
  valid_prediction = model.predict(x=[vx_enc[:,:-1],vx_enc[:,-1]])
  #TEST RESULTS
  prediction_after_scale = scaler.inverse_transform(test_prediction)
  true_values = scaler.inverse_transform(tey_scaled)
  test_r2 = r2_score(true_values,prediction_after_scale)
  test_mae = mean_absolute_error(true_values,prediction_after_scale)

  #TRAIN RESULTS
  prediction_after_scale = scaler.inverse_transform(train_prediction)
  true_values = scaler.inverse_transform(y_scaled)
  train_r2 = r2_score(true_values,prediction_after_scale)
  train_mae = mean_absolute_error(true_values,prediction_after_scale)

  #VALIDATION RESULTS
  prediction_after_scale = scaler.inverse_transform(valid_prediction)
  true_values = scaler.inverse_transform(vy_scaled)
  valid_r2 = r2_score(true_values,prediction_after_scale)
  valid_mae = mean_absolute_error(true_values,prediction_after_scale)
  return (train_r2,valid_r2,test_r2) , (train_mae,valid_mae,test_mae)

In [ ]:
results_without = {}
for col in [None,'Sheeting_Type','Color','Orientation_(Degrees)','Observation_Angle_(Degrees)']:
  for m in [('ohe' , exp_ohe), ('linear', exp_linear),('poly',exp_poly),('sharedEmbed',exp_sharedEmbed),('sepEmbed',exp_sepEmbed)]:
    name , exp_mod = m
    train_r2 = []
    train_mae = []
    valid_r2 = []
    valid_mae = []
    test_r2 = []
    test_mae = []
    for i in range(10):
      r2, mae = exp_mod(col)
      t , v, te = r2
      train_r2.append(t)
      valid_r2.append(v)
      test_r2.append(te)
      t , v , te = mae
      train_mae.append(t)
      valid_mae.append(v)
      test_mae.append(te)
    train_r2_results = str(np.mean(train_r2) ) + '±'+ str(np.std(train_r2))
    train_mae_results = str(np.mean(train_mae) ) + '±'+ str(np.std(train_mae))
    valid_r2_results = str(np.mean(valid_r2) ) + '±'+ str(np.std(valid_r2))
    valid_mae_results = str(np.mean(valid_mae) ) + '±'+ str(np.std(valid_mae))
    test_r2_results = str(np.mean(test_r2) ) + '±'+ str(np.std(test_r2))
    test_mae_results = str(np.mean(test_mae) ) + '±'+ str(np.std(test_mae))
    if col == None:
      results_without['all'] = {'name' : name, 'train_r2' : train_r2_results,'train_mae' : train_mae_results,
                                'valid_r2' : valid_r2_results, 'valid_mae' : valid_mae_results,
                                'test_r2' : test_r2_results, 'test_mae' : test_mae_results}
    else:
      results_without[col] = {'name' : name, 'train_r2' : train_r2_results,'train_mae' : train_mae_results,
                                'valid_r2' : valid_r2_results, 'valid_mae' : valid_mae_results,
                                'test_r2' : test_r2_results, 'test_mae' : test_mae_results}

In [ ]:
import json
with open("results.json", "w") as outfile:
    json.dump(results_without, outfile,indent=4)

In [1]:
import json

with open("results.json", "r") as infile:
    x = json.load(infile)

In [2]:
x

{'all': {'name': 'sepEmbed',
  'train_r2': '0.033146659319700164±0.4608042540338364',
  'train_mae': '154.1158±57.803238',
  'valid_r2': '0.013907833525415692±0.4104680848528115',
  'valid_mae': '200.9431±47.181885',
  'test_r2': '0.04395483680008565±0.45931889352632055',
  'test_mae': '153.2675±57.534573'},
 'Sheeting_Type': {'name': 'sepEmbed',
  'train_r2': '0.5301517981127136±0.003886210519734137',
  'train_mae': '129.28708±1.1719836',
  'valid_r2': '0.7096173969307638±0.00883210334663149',
  'valid_mae': '143.70761±5.1966963',
  'test_r2': '0.7399618084730291±0.018810458338285676',
  'test_mae': '85.939575±2.2473667'},
 'Color': {'name': 'sepEmbed',
  'train_r2': '0.3984964577185383±0.2165092123946608',
  'train_mae': '143.05386±9.915717',
  'valid_r2': '0.4716480053827798±0.24895640007902667',
  'valid_mae': '175.72849±23.827394',
  'test_r2': '0.09657925154477516±0.06826012735511487',
  'test_mae': '198.68674±18.49206'},
 'Orientation_(Degrees)': {'name': 'sepEmbed',
  'train_r2